## <a id = '0'> Índice </a>

* [**Entorno**](#1)  
   * [Librerías](#1d1)  
   * [Funciones](#1d2)  
   * [Constantes](#1d3)

* [**Lectura de datos**](#2)


## <a id = '1'> Entorno </a>
[índice](#0)

### <a id = '1d1'> Librerías </a>

In [1]:
import os
import pandas as pd
import numpy as np

import math
import itertools
# from config import data_folder

from scipy.stats import kurtosis, trim_mean


In [2]:
os.chdir("../")

### <a id = '1d2'> Funciones </a>

In [3]:
from src.utils import (lectura_carpetas_dict
                       , split_dataframe_into_chunks
                       , genera_df_acf_pacf_chunk
                       , get_dict_serie_summary_chunk
                       , patients_dict_ccf_chunk
                       , get_ccf_summary_chunk
                       )

### <a id = '1d3'> Constantes </a>

In [4]:
from config.cons import data_folder, list_signals

## <a id = '2'> Lectura de datos </a>
[índice](#0)

In [5]:
proyect_path = os.getcwd()
data_path = proyect_path + data_folder

In [6]:
# all_combinaciones = list(itertools.combinations(list_signals, 2))
all_combinaciones = list(itertools.combinations_with_replacement(list_signals, 2))


In [7]:
uni_combinacion = []
bi_combinacion = []
for col1, col2 in all_combinaciones:
    if col1 == col2:
        uni_combinacion.append(col1)
    else:
        bi_combinacion.append(col1 + "_" + col2)
dict_combinaciones = {
    "uni_combinacion": uni_combinacion,
    "bi_combinacion": bi_combinacion
}   

In [8]:
df_all = lectura_carpetas_dict(data_path)

In [9]:
df_mi = df_all["mi"]
df_sttc_mi = df_all["sttc_mi"]
df_sttc = df_all["sttc"]
df_other = df_all["other"]

In [10]:
dict_mi = {patient: split_dataframe_into_chunks(df_mi[patient], 100) for patient in df_mi.keys()}
df_mi_acf_pacf = genera_df_acf_pacf_chunk(dict_mi, list_signals, apply_diff=True)
df_mi_peak = get_dict_serie_summary_chunk(dict_mi, list_signals, period = 10, n_std = 1)
df_mi_ccf = patients_dict_ccf_chunk(dict_mi, all_combinaciones, max_lag=25)
mi_cff_features = get_ccf_summary_chunk(df_mi_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [11]:
df_mi_features = (
    df_mi_acf_pacf
    .merge(df_mi_peak, on=["patient_id", "label"], how="inner")
    .merge(mi_cff_features, on=["patient_id", "label"], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_mi_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_mi_features = df_mi_features[cols]

In [13]:
df_mi_features.to_csv("output/chunk_data/chunk_100/features/mi_features.csv", index=False)

In [14]:
del dict_mi, df_mi_acf_pacf, df_mi_peak, mi_cff_features

In [15]:
dict_sttc_mi = {patient: split_dataframe_into_chunks(df_sttc_mi[patient], 100) for patient in df_sttc_mi.keys()}
df_sttc_mi_acf_pacf = genera_df_acf_pacf_chunk(dict_sttc_mi, list_signals, apply_diff=True)
df_sttc_mi_peak = get_dict_serie_summary_chunk(dict_sttc_mi, list_signals, period = 10, n_std = 1)
df_sttc_mi_ccf = patients_dict_ccf_chunk(dict_sttc_mi, all_combinaciones, max_lag=25)
sttc_mi_cff_features = get_ccf_summary_chunk(df_sttc_mi_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [16]:
df_sttc_mi_features = (
    df_sttc_mi_acf_pacf
    .merge(df_sttc_mi_peak, on=["patient_id", "label"], how="inner")
    .merge(sttc_mi_cff_features, on=["patient_id", "label"], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_sttc_mi_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_sttc_mi_features = df_sttc_mi_features[cols]

In [17]:
del dict_sttc_mi, df_sttc_mi_acf_pacf, df_sttc_mi_peak, df_sttc_mi_ccf

In [18]:
df_sttc_mi_features.to_csv("output/chunk_data/chunk_100/features/sttc_mi_features.csv", index=False)


In [19]:
dict_sttc = {patient: split_dataframe_into_chunks(df_sttc[patient], 100) for patient in df_sttc.keys()}
df_sttc_acf_pacf = genera_df_acf_pacf_chunk(dict_sttc, list_signals, apply_diff=True)
df_sttc_peak = get_dict_serie_summary_chunk(dict_sttc, list_signals, period = 10, n_std = 1)
df_sttc_ccf = patients_dict_ccf_chunk(dict_sttc, all_combinaciones, max_lag=25)
sttc_cff_features = get_ccf_summary_chunk(df_sttc_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [20]:
df_sttc_features = (
    df_sttc_acf_pacf
    .merge(df_sttc_peak, on=["patient_id", "label"], how="inner")
    .merge(sttc_cff_features, on=["patient_id", "label"], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_sttc_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_sttc_features = df_sttc_features[cols]

In [21]:
df_sttc_features.to_csv("output/chunk_data/chunk_100/features/sttc_features.csv", index=False)

In [22]:
del dict_sttc, df_sttc_acf_pacf, df_sttc_peak, sttc_cff_features

In [23]:
dict_other = {patient: split_dataframe_into_chunks(df_other[patient], 100) for patient in df_other.keys()}
df_other_acf_pacf = genera_df_acf_pacf_chunk(dict_other, list_signals, apply_diff=True)
df_other_peak = get_dict_serie_summary_chunk(dict_other, list_signals, period = 10, n_std = 1)
df_other_ccf = patients_dict_ccf_chunk(dict_other, all_combinaciones, max_lag=25)
other_cff_features = get_ccf_summary_chunk(df_other_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [24]:
df_other_features = (
    df_other_acf_pacf
    .merge(df_other_peak, on=["patient_id", "label"], how="inner")
    .merge(other_cff_features, on=["patient_id", "label"], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_other_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_other_features = df_other_features[cols]

In [25]:
df_other_features.to_csv("output/chunk_data/chunk_100/features/other_features.csv", index=False)

In [26]:
del dict_other, df_other_acf_pacf, df_other_peak, other_cff_features

In [ ]:
df_mi_features = pd.read_csv("output/chunk_data/chunk_100/features/mi_features.csv")
df_sttc_mi_features = pd.read_csv("output/chunk_data/chunk_100/features/sttc_mi_features.csv")
df_sttc_features = pd.read_csv("output/chunk_data/chunk_100/features/sttc_features.csv")
df_other_features = pd.read_csv("output/chunk_data/chunk_100/features/other_features.csv")

In [27]:
patients = pd.read_csv("output/split_train_test/patients_90_05_05.csv").rename(columns={"patient": "patient_id", "class": "label"})

In [28]:
patients_train = patients[(patients["sample"] == "train")][["patient_id", "label"]]
patients_test = patients[(patients["sample"] == "test")][["patient_id", "label"]]
patients_val = patients[(patients["sample"] == "validation")][["patient_id", "label"]]

In [29]:
df_features = pd.concat([df_mi_features, df_sttc_mi_features, df_sttc_features, df_other_features], axis=0)

In [34]:
df_features.to_csv("output/chunk_data/chunk_100/features/all_features.csv", index=False)

In [31]:
del df_mi_features, df_sttc_mi_features, df_sttc_features, df_other_features

In [ ]:
df_features = pd.read_csv("output/chunk_data/chunk_100/features/all_features.csv")

In [32]:
features_train = df_features.merge(patients_train, on=["patient_id", "label"], how="inner")

In [35]:
features_train.to_csv("output/chunk_data/chunk_100/train_test_val/train_features.csv", index=False)

In [36]:
features_test = df_features.merge(patients_test, on=["patient_id", "label"], how="inner")

In [37]:
features_test.to_csv("output/chunk_data/chunk_100/train_test_val/test_features.csv", index=False)

In [38]:
features_val = df_features.merge(patients_val, on=["patient_id", "label"], how="inner")

In [39]:
features_val.to_csv("output/chunk_data/chunk_100/train_test_val/val_features.csv", index=False)

### Train test val 80 - 10 - 10

In [11]:
patients = pd.read_csv("output/split_train_test/patients_80_10_10.csv").rename(columns={"patient": "patient_id", "class": "label"})

In [12]:
patients_train = patients[(patients["sample"] == "train")][["patient_id", "label"]]
patients_test = patients[(patients["sample"] == "test")][["patient_id", "label"]]
patients_val = patients[(patients["sample"] == "validation")][["patient_id", "label"]]

In [13]:
df_features = pd.read_csv("output/chunk_data/chunk_100/features/all_features.csv")

In [14]:
features_train = df_features.merge(patients_train, on=["patient_id", "label"], how="inner")
features_train.to_csv("output/chunk_data/chunk_100/train_test_val/train_features.csv", index=False)

In [15]:
features_train.shape

(1919000, 834)

In [16]:
features_test = df_features.merge(patients_test, on=["patient_id", "label"], how="inner")
features_test.to_csv("output/chunk_data/chunk_100/train_test_val/test_features.csv", index=False)

In [17]:
features_val = df_features.merge(patients_val, on=["patient_id", "label"], how="inner")
features_val.to_csv("output/chunk_data/chunk_100/train_test_val/val_features.csv", index=False)